# About: scpによるリストア

---

Moodle構築環境のデータ、設定ファイルなどのバックアップをscpを利用してリストアします。

## 概要

scpを利用してMoodle環境のリストアを行います。

### 前提条件


この Notebook を実行するには事前に以下のものを準備する必要があります。

* リストア対象のホストからバックアップ保存先のホストにSSH公開鍵認証でログインできること
* リストア先となるVCノード/EC2インスタンス/Azure仮想マシンが作成済であること

リストア先となる環境は「011-VCノードの作成」、「012-EC2インスタンスの作成」、「013-Azure仮想マシンの作成」のいずれかのNotebookで作成することができます。

また、リストアの操作により既存の環境を損ねることを避けるため、リストア先となるディレクトリ `/srv/moodle` が存在していないことを前提としています。

## パラメータ設定

### リストア先のMoodle環境を指定する

リストア先となるノードをAnsibleのグループ名で指定してください。

In [ ]:
# (例)
# target_group = 'Moodle'

target_group =

リストア対象のノードにアクセスできることを確認します。

In [ ]:
!ansible {target_group} -m ping

リストア先となるディレクトリ `/srv/moodle` が存在していないことを確認します。

In [ ]:
!ansible {target_group} -a 'test ! -e /srv/moodle'

### SSH公開鍵認証

バックアップの保存先にSSHでアクセスするためのパラメータを指定します。

バックアップ先のホストにログインする際のユーザ名を指定してください。

In [ ]:
# (例)
# backup_user = 'user01'

backup_user =

バックアップ先のホスト名を指定してください。

In [ ]:
# (例)
# backup_host = 'backup.example.org'

backup_host =

バックアップ先のホストにログインする際のSSHの秘密鍵のパスを指定してください。ここで指定するパスはMoodle構築環境におけるパスを指定する必要があります。

In [ ]:
# (例)
# backup_ssh_identity = '~/.ssh/id_rsa'

backup_ssh_identity =

指定されたパスに秘密鍵のファイルが存在していることをチェックします。次のセルを実行してエラーにならないことを確認してください。

In [ ]:
!ansible {target_group} -m shell -a 'test -f {backup_ssh_identity}'

SSHの公開鍵ペアファイルをまだ作成していない場合は、次のセルのコメント `#` を外し実行することで公開鍵認証のファイルを作成することができます。

In [ ]:
# !ansible {target_group} -m shell -a \
#     'test -f {backup_ssh_identity} || \
#     ssh-keygen -q -t rsa -N "" -f {backup_ssh_identity} \
#     && cat {backup_ssh_identity}.pub'

バックアップ先のホストにログインする前に `~/.ssh/known_hosts` を更新しておきます。

> 既に `~/.ssh/known_hosts` にバックアップ先のホストを登録してある場合は次のセルの実行をスキップしてください。

In [ ]:
!ansible {target_group} -m shell -a \
    'ssh-keyscan {backup_host} >> ~/.ssh/known_hosts'

バックアップ先のホストにSSHでログインしてコマンド `ls -la` を実行してみます。

In [ ]:
ssh_command = f'ssh -i {backup_ssh_identity} {backup_user}@{backup_host}'

!ansible {target_group} -m shell -a \
    '{ssh_command} ls -la'

### リストアを行うバックアップファイルを指定する

バックアップファイルの保存先であるディレクトリを指定してください。

In [ ]:
# (例)
# backup_dir = 'moodle-simple/Moodle/2020-XX-XXTXX:XX:XX.XXXXXX'

backup_dir =

指定したバケット名、ディレクトリにバックアップファイルがあることを確認します。次のセルを実行してMoodle環境のバックアップファイル `db.sql.gz`, `moodle.tar.gz` が存在していることを確認してください。

In [ ]:
!ansible {target_group} -m shell -a \
    '{ssh_command} ls -la {backup_dir}'

## リストア

Moodle環境のリストアを行います。

### ディレクトリの作成

リストア先となるディレクトリを作成します。

In [ ]:
!ansible {target_group} -b -m file -a \
    'path=/srv/moodle state=directory owner={{{{ansible_ssh_user}}}}'

デフォルトのコンテナログの出力先ディレクトリを作成します。

In [ ]:
!ansible {target_group} -b -m file -a 'path=/var/log/httpd state=directory'
!ansible {target_group} -b -m file -a 'path=/var/log/mysql owner=999 group=adm state=directory'

### コンテナ構成、Moodleデータなどのリストア

設定ファイルなどのバックアップファイル `moodle.tar.gz` をリストアします。

リストア対象のバックアップファイルを確認します。

In [ ]:
moodle_backup = backup_dir + '/moodle.tar.gz'
print(moodle_backup)

リストアを行います。

In [ ]:
!ansible {target_group} -m shell -a \
    '{ssh_command} cat {moodle_backup} \
    | bash -c "sudo tar xzpf - -C /srv/moodle"'

リストアされたことを確認します。

In [ ]:
!ansible {target_group} -a 'chdir=/srv/moodle \
    tree -L 3 -F'

### DBデータ

DBデータを復元するためのSQLファイルを取得します。

リストア対象のバックアップファイルを確認します。

In [ ]:
db_backup = backup_dir + '/db.sql.gz'
print(db_backup)

リストア先となるディレクトリを作成します。

In [ ]:
!ansible {target_group} -b -m file -a \
    'path=/srv/moodle/db/data state=directory owner={{{{ansible_ssh_user}}}}'
!ansible {target_group} -b -m file -a \
    'path=/srv/moodle/db/sql state=directory owner={{{{ansible_ssh_user}}}}'

DBデータをリストアするためのSQLファイルファイルを配置します。配置したSQLファイルはDBコンテナ起動時に自動的に実行されます。

In [ ]:
!ansible {target_group} -m shell -a \
    '{ssh_command} cat {db_backup} > /srv/moodle/db/sql/backup.sql.gz'

### コンテナイメージ

Dockerコンテナイメージのリストアを行います。

> コンテナイメージのバックアップを作成していない、あるいはレポジトリのコンテナイメージを利用する場合はこの節を実行せずにスキップしてください。

コンテナイメージをリストアする前の状態を確認しておきます。コンテナイメージの一覧を表示します。

In [ ]:
!ansible {target_group} -a 'docker images'

リストア対象のバックアップファイルを確認します。

In [ ]:
img_backup = backup_dir + '/container-image.tar.gz'
print(img_backup)

コンテナイメージをリストアします。

> バックアップファイルが存在していない場合はエラーになります。

In [ ]:
!ansible {target_group} -m shell -a \
    '{ssh_command} cat {img_backup} | \
    gzip -cd | docker load'

リストアを行った後の状態を確認します。コンテナイメージの一覧を表示します。

In [ ]:
!ansible {target_group} -a 'docker images'

### コンテナの起動

リストア環境のコンテナを起動します。

In [ ]:
!ansible {target_group} -a 'chdir=/srv/moodle \
    docker-compose up -d'

コンテナの状態を確認します。`State`が`Up`となっていることを確認してください。

In [ ]:
!ansible {target_group} -a 'chdir=/srv/moodle \
    docker-compose ps'

### crontab

crontabの設定を復元します。

現在の crontab の設定を確認します。

In [ ]:
try:
    !ansible {target_group} -a 'crontab -l'
except:
    pass

バックアップファイルからcrontabの復元を行います。

In [ ]:
!ansible {target_group} -a 'crontab /srv/moodle/misc/crontab'

復元後の設定を確認します。

In [ ]:
!ansible {target_group} -a 'crontab -l'

### logrotate

コンテナログのログローテーションの設定を復元します。

In [ ]:
!ansible {target_group} -b -m shell -a \
    'cp /srv/moodle/misc/logrotate.d/* /etc/logrotate.d/'

### メンテナンスモードの解除

メンテナンスモードに変更してからバックアップを作成した場合はメンテナンスモードの解除が必要となります。

In [ ]:
!ansible {target_group} -a 'chdir=/srv/moodle docker-compose exec -T moodle \
    /usr/bin/php /var/www/html/admin/cli/maintenance.php --disable'